In [31]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
#df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV.json")
df = sqlContext.read.json("s3n://amazon-reviews/complete.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [32]:
ratings = df
ratings = ratings.drop("helpful")
ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin', 'overall', 'reviewerID']

In [33]:
intmax = 2 ** 31 - 1
rawRatings3 = ratings.map(lambda row: (abs(hash(row.reviewerID))%intmax, abs(hash(row.asin))%intmax, row.overall))

In [34]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [35]:
rank = 5
numIterations = 5
model = ALS.train(rawRatings3, rank, numIterations)
#model = ALS.train(rawRatings3, nonnegative=True, seed=10)

Py4JJavaError: An error occurred while calling o159.trainALSModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 447.2 failed 4 times, most recent failure: Lost task 13.3 in stage 447.2 (TID 8807, 172.31.45.95): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.PythonRDD$$anon$1.read(PythonRDD.scala:175)
	at org.apache.spark.api.python.PythonRDD$$anon$1.<init>(PythonRDD.scala:179)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:97)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:277)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:244)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:277)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:244)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin

In [ ]:
model.recommendProducts(8751,5)

In [ ]:
model.userFeatures().take(5)

In [ ]:
import os, tempfile
path = "s3n://patricks3db/modelsComplete"
model.save(sc, path)

In [ ]:
print path

In [ ]:
sameModel = MatrixFactorizationModel.load(sc, path)

In [ ]:
sameModel.recommendProducts(8751,5)